In [6]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import os

In [7]:
data_path = '../data/'
files = []

#lists the total number of .wav files in /data/
for f in os.listdir(data_path):
    if f.endswith('.wav'):
        files.append(f)

print(f"Found {len(files)} .wav files. Preparing analysis.")

Found 7 .wav files. Preparing analysis.


In [8]:
#calculates the spectral centroid, spectral flatness and zero crossing rate for all files in /data/
for file_name in files:
    full_path = os.path.join(data_path, file_name)
    y, sr = librosa.load(full_path)
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    flat = librosa.feature.spectral_flatness(y=y)[0]
    zero = librosa.feature.zero_crossing_rate(y=y)[0]

    #calculates 'average brightness'
    brightness = np.mean(cent)
    #calculates 'average flatness'
    avg_flat = np.mean(flat*100)
    #calculates 'average smoothness'
    smooth = np.mean(100 - (zero*100))

    print(f"File: {file_name}")
    print(f"Average brightness: {brightness:.2f} hz")
    print(f"Average flatness: {avg_flat:.2f}%")
    print(f"Average smoothness: {smooth:.2f}%")
    print('---------------------------------')








File: Meow.wav
Average brightness: 1819.82 hz
Average flatness: 0.31%
Average smoothness: 90.61%
---------------------------------
File: Low Drone.wav
Average brightness: 160.00 hz
Average flatness: 0.03%
Average smoothness: 98.83%
---------------------------------
File: Ringing.wav
Average brightness: 5854.18 hz
Average flatness: 13.16%
Average smoothness: 41.05%
---------------------------------
File: Glass Shatter.wav
Average brightness: 5605.30 hz
Average flatness: 18.47%
Average smoothness: 51.87%
---------------------------------
File: Synth.wav
Average brightness: 2993.20 hz
Average flatness: 2.93%
Average smoothness: 86.71%
---------------------------------
File: Car Breaking.wav
Average brightness: 2561.40 hz
Average flatness: 2.44%
Average smoothness: 87.23%
---------------------------------
File: Racquet Hit.wav
Average brightness: 2239.55 hz
Average flatness: 1.51%
Average smoothness: 88.34%
---------------------------------
